In [1]:
import matplotlib.pyplot as plt
from matplotlib.pyplot import ion
import matplotlib.dates as mdates
%matplotlib tk
import numpy as np
from poliastro.bodies import Sun
from poliastro.core.iod import izzo
from datetime import datetime
from tqdm import tqdm
from scipy.ndimage import binary_dilation
import matplotlib.font_manager as fm
from lamberthub import izzo2015,vallado2013

In [2]:
targets = ['SUN', 'EARTH', 'MARS', 'SATELLITE']

fig = plt.figure()
ax = fig.add_subplot(projection='3d')

ax.set_xlabel('X (AU)')
ax.set_ylabel('Y (AU)')
ax.set_zlabel('Z (AU)')
ax.set_title('Orbit in AU Space')

all_px, all_py, all_pz = [], [], []

for target in targets:
    try :
        data = np.genfromtxt(f'simulation_results/{target}.csv', delimiter=',', skip_header=1)
    except :
        data = np.genfromtxt(f'traces/{target}.csv', delimiter=',', skip_header=1)
        
    px, py, pz = data[:, 1], data[:, 2], data[:, 3]
    
    all_px.extend(px)
    all_py.extend(py)
    all_pz.extend(pz)
    
    ax.plot(px, py, pz, label=target)  # Add label for better visualization

# Compute overall range
all_px, all_py, all_pz = np.array(all_px), np.array(all_py), np.array(all_pz)

max_range = max(all_px.ptp(), all_py.ptp(), all_pz.ptp()) / 2.0
mid_x = (all_px.max() + all_px.min()) / 2.0
mid_y = (all_py.max() + all_py.min()) / 2.0
mid_z = (all_pz.max() + all_pz.min()) / 2.0

# Set the global limits
ax.set_xlim(mid_x - max_range, mid_x + max_range)
ax.set_ylim(mid_y - max_range, mid_y + max_range)
ax.set_zlim(mid_z - max_range, mid_z + max_range)

ax.legend()  # Show labels

#plt.show()
print('done')

3

In [3]:
def load_data(host : str, target : str) :
    
    try :
        dates = np.genfromtxt(f'simulation_results\{host}.csv',delimiter=',',usecols=[0],skip_header=True,dtype=str) 

        host_positions = np.genfromtxt(f'simulation_results\{host}.csv',delimiter=',',usecols=[1,2,3],skip_header=True) 
        host_velocities = np.genfromtxt(f'simulation_results\{host}.csv',delimiter=',',usecols=[4,5,6],skip_header=True)

        target_positions = np.genfromtxt(f'simulation_results\{target}.csv',delimiter=',',usecols=[1,2,3],skip_header=True) 
        target_velocities = np.genfromtxt(f'simulation_results\{target}.csv',delimiter=',',usecols=[4,5,6],skip_header=True)  
    
    except : 
        dates = np.genfromtxt(f'traces\{host}.csv',delimiter=',',usecols=[0],skip_header=True,dtype=str) 

        host_positions = np.genfromtxt(f'traces\{host}.csv',delimiter=',',usecols=[1,2,3],skip_header=True) 
        host_velocities = np.genfromtxt(f'traces\{host}.csv',delimiter=',',usecols=[4,5,6],skip_header=True)

        target_positions = np.genfromtxt(f'traces\{target}.csv',delimiter=',',usecols=[1,2,3],skip_header=True) 
        target_velocities = np.genfromtxt(f'traces\{target}.csv',delimiter=',',usecols=[4,5,6],skip_header=True)  
        
    return dates, host_positions, host_velocities, target_positions, target_velocities

In [4]:
dates, host_positions, host_velocities, target_positions, target_velocities = load_data('EARTH_TRACE','MARS_TRACE')
dates = [datetime.strptime(date, "%A %B %d %Y %H:%M:%S").timestamp() for date in dates]
dates = np.array(dates)
t0 = dates[0]
dates = dates - t0

elements = dates.shape[0] 
resolution = 0.0001
interval = elements / (elements * resolution)

launch_from  = (0) *24*60*60 # 605 days from 2025-00-00 
launch_until = (4*365) *24*60*60 # 315
launch_times = np.extract(dates >= launch_from,dates)
launch_times = np.extract(dates <= launch_until,launch_times)

mask = np.isin(dates, launch_times)
launch_positions = host_positions[mask]
launch_velocities = host_velocities[mask]

arrive_after  = 100 *24*60*60 # 100
arrive_before = (1000) *24*60*60 #-5

lower_bound = min(launch_times) + arrive_after
upper_bound = max(launch_times) + arrive_before 

arrival_times = np.extract(dates >= lower_bound,dates)
arrival_times = np.extract(dates <= upper_bound,arrival_times)

mask = np.isin(dates, arrival_times)
arrival_positions = target_positions[mask]
arrival_velocities = target_velocities[mask]

launch_times += t0
arrival_times += t0

In [5]:
ax = plt.figure().add_subplot(projection='3d')
ax.plot(host_positions[:,0],host_positions[:,1],host_positions[:,2],color='steelblue')
ax.plot(launch_positions[:,0],launch_positions[:,1],launch_positions[:,2],color='red')

ax.plot(target_positions[:,0],target_positions[:,1],target_positions[:,2],color='steelblue')
ax.plot(arrival_positions[:,0],arrival_positions[:,1],arrival_positions[:,2],color='red')

a = np.argmin(launch_times)
b = np.argmax(launch_times)
ax.scatter(launch_positions[a][0],launch_positions[a][1],launch_positions[a][2],color='steelblue')
ax.scatter(launch_positions[b][0],launch_positions[b][1],launch_positions[b][2],color='red')

In [6]:
final = []
c = 0
step = 100 # Change step size to control skipping rate
total_iterations = len(launch_positions[::step]) * len(target_positions[::step])

xs = np.array(launch_times[::step])
ys = np.array(arrival_times[::step])

tofs = np.zeros(len(xs)*len(ys))
v_is = np.zeros(len(xs)*len(ys))
v_fs = np.zeros(len(xs)*len(ys))

count = 0

with tqdm(total=total_iterations, desc="Processing", unit="iter") as pbar:
    for lpos, lt, lv in zip(launch_positions[::step], launch_times[::step], launch_velocities[::step]):
        for tpos, tt, tv in zip(arrival_positions[::step], arrival_times[::step], arrival_velocities[::step]):
            
            tof = tt - lt
        
            tofs[count] = tof
            
            try :
                v_i, v_f = izzo(Sun.k, lpos, tpos, tof, 0, True, False, 150, 1e-8)
                
                if np.linalg.norm((v_i-lv)/1000)**2 <=20 :  
                    v_is[count] = np.linalg.norm((v_i-lv)/1000)**2
                else :
                    v_is[count] = np.nan
                    
                
                    
            except:
                v_is[count] = np.nan

            count +=1
            pbar.update(1)  # Update progress bar for each iteration

Processing:  73%|███████▎  | 1461376/1996344 [00:14<00:05, 101423.78iter/s]


In [9]:
monospace_font = {'fontname': 'Courier New', 'fontsize': 14, 'weight': 'bold'}

xs = [datetime.fromtimestamp(t) for t in launch_times[::step]]
ys = [datetime.fromtimestamp(t) for t in arrival_times[::step]]

XS, YS = np.meshgrid(xs, ys)

Z = v_is.reshape((len(xs), len(ys)))
Z2 = tofs.reshape((len(xs), len(ys)))

buffer_size = 6
Z_mask = np.isnan(Z)
expanded_mask = binary_dilation(~Z_mask, iterations=buffer_size)
Z2_masked = np.ma.masked_where(~expanded_mask, Z2 / (24 * 60 * 60))  # Apply the buffer

# Create figure and axis
fig, ax = plt.subplots(figsize=(10,8))
ax.set_axisbelow(True)
# Set limits
launch_start_date = datetime(2026, 8, 1)
launch_end_date = datetime(2027, 3, 1)

arrival_start_date = datetime(2026, 7, 1)
arrival_end_date = datetime(2028, 7, 1)

#ax.set_xlim(launch_start_date, launch_end_date)
#ax.set_ylim(arrival_start_date, arrival_end_date)

# Contour plots
ax.contour(XS, YS, Z.T, colors='k',alpha=0.15)
c_vi = ax.contourf(XS, YS, Z.T)
c_tofs = ax.contour(XS, YS, Z2_masked.T, levels=15, colors='black',linestyles='dashed',zorder=10)

ax.clabel(c_tofs, fmt=lambda x: f"{x:.1f}", inline=True, fontsize=8)

cbar = fig.colorbar(c_vi, ax=ax)
cbar.set_label(r'Required $\Delta$V Impulse Burn [km/s]',fontproperties=fm.FontProperties(family="Courier New", size=10))
for label in cbar.ax.get_yticklabels():
    label.set_fontproperties(fm.FontProperties(family="Courier New", size=10))
    
# Set minor and major ticks
ax.xaxis.set_minor_locator(mdates.DayLocator(interval=7))
ax.yaxis.set_minor_locator(mdates.DayLocator(interval=15))

ax.tick_params(axis='x', which='minor', length=2, color='gray')
ax.tick_params(axis='y', which='minor', length=2, color='gray')

ax.yaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
ax.yaxis.set_major_locator(mdates.MonthLocator(interval=5))
ax.set_yticks(ax.get_yticks())  
plt.setp(ax.get_yticklabels(), fontproperties=fm.FontProperties(family="Courier New"), fontsize=7, rotation=45)  

ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
ax.xaxis.set_major_locator(mdates.MonthLocator(interval=1))
ax.set_xticks(ax.get_xticks())  
plt.setp(ax.get_xticklabels(), fontproperties=fm.FontProperties(family="Courier New"), fontsize=7, rotation=45)  

# Labels
ax.set_xlabel('Launch Date',fontproperties=fm.FontProperties(family="Courier New"), fontsize=12)
ax.set_ylabel('Arrival Date',fontproperties=fm.FontProperties(family="Courier New"), fontsize=12)

plt.suptitle("EARTH TO MARS 2005   type 1,2\nTTIME[ --- black] , C3[CONTOUR]\nBallistic transfer solution", 
             fontproperties=fm.FontProperties(family="Courier New"), fontsize=14, fontweight='bold')

plt.grid()
plt.show()